**Data Cleaning**
1. import packages

    numpy -- array number manipulation
    
    pandas -- data storage and other number manipulation stuff
    
    sklearn -- machine learning stuff(kernel ridge regression, split data into testing and training data, calculating accuracy(explained variance score))

2. pull data from the github and set up dataframe

3. standardardize the data(go from random scales in each column to have data based on standard deviations(roughly from 0-10))

4. fill in missing values in the data with a reasonable estimate(in this case, the average value across all counties)

In [ ]:
#import and clean data
#importing packages
import warnings
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils
import collections

#set up the dataframe
#data is partially cleaned
#pull the data from github
#todo: add infant mortality(LOL health score metric) and living wage(neghealthmetriclist)
data = pd.read_csv('https://raw.githubusercontent.com/Charliehyin/healthcare-resource-optimizer/main/county%20data%20-%20analytic_data2022.csv', header=None)
#setup column names for the data
data.columns = ['State Code', 'County Code', 'FIPS Code', 'State Abbreviation', 'County Name', 'Premature Death', 
                'Poor or Fair Health', 'Poor Physical Health Days', 'Poor Mental Health Days', 
                'Low Birthweight', 'Adult Smoking', 'Adult Obesity', 'Food Environment Index', 'Physical Inactivity', 
                'Access to Exercise Opportunities', 'Excessive Drinking', 'Alcohol-Impaired Driving Deaths', 'Sexually-Transmitted Infections', 'Teen Births', 
                'Uninsured', 'Primary Care Physicians', 'Dentists', 'Mental Health Providers', 'Preventable Hospital Stays', 
                'Mammography Screening', 'Flu Vaccinations', 'High School Completion', 'Some College', 'Unemployment', 
                'Children in Poverty', 'Income Inequality', 'Children in Single-Parent Households', 'Social Associations', 
                'Violent Crime', 'Injury Deaths', 'Air Pollution - Particulate Matter', 'Drinking Water Violations', 
                'Severe Housing Problems', 'Percentage of Households With High Housing Costs', 'Driving Alone to Work', 
                'Long Commute - Driving Alone', 'COVID-19 Age-Adjusted Mortality', 'Life Expectancy', 'Premature Age-Adjusted Mortality', 
                'Child Mortality', 'Frequent Physical Distress', 'Frequent Mental Distress', 'Diabetes Prevalence', 'HIV Prevalence', 'Food Insecurity', 
                'Limited Access to Healthy Foods', 'Drug Overdose Deaths', 'Motor Vehicle Crash Deaths', 'Insufficient Sleep', 
                'Uninsured Adults', 'Uninsured Children', 'Other Primary Care Providers', 'High School Graduation', 'Disconnected Youth', 
                'Reading Scores', 'Reading Scores(Black)', 'Reading Scores(Hispanic)', 'Reading Scores(White)', 'Math Scores', 
                'School Segregation', 'School Funding Adequacy', 'Gender Pay Gap', 'Median Household Income', 'Children Eligible for Free or Reduced Price Lunch', 
                'Residential Segregation - Black/White', 'Residential Segregation - Non-White/White', 'Childcare Cost Burden', 
                'Childcare Centers', 'Homicides', 'Suicides', 'Firearm Fatalities', 'Juvenile Arrests', 'Traffic Volume', 'Homeownership', 
                'Severe Housing Cost Burden', 'Broadband Access', 'Population', 'Percent Below 18 Years of Age', 'Percent 65 and Older', 
                'Percent Non-Hispanic Black', 'Percent American Indian & Alaska Native', 'Percent Asian', 'Percent Native Hawaiian/Other Pacific Islander', 
                'Percent Hispanic', 'Percent Non-Hispanic White', 'Percent Not Proficient in English', 'Percent Female', 'Percent Rural']    
data = data.drop(labels=0, axis=0)
df = pd.DataFrame(data)

#function to standardize all numerical columns
def standardize(df):
  result = df.copy()
  for feature_name in df.columns:
    #dont standardize these columns
    if feature_name != 'State Abbreviation' and feature_name != 'County Name' and feature_name != 'State Code' and feature_name != 'FIPS Code' and feature_name != 'County Code':
      #convert column to numeric
      result[feature_name] = pd.to_numeric(df[feature_name])
      #standardize data
      max_value = result[feature_name].max()
      min_value = result[feature_name].min()
      #this line is the calculation
      result[feature_name] = (result[feature_name]-result[feature_name].mean())/result[feature_name].std()
      #shift data up so it's not negative
      new_min_value = result[feature_name].min()
      result[feature_name] = result[feature_name]-new_min_value
  return result

df = standardize(df)

#fill in all empty items with means
for feature_name in df.columns:
  #dont look at these columns
  if feature_name != 'State Abbreviation' and feature_name != 'County Name' and feature_name != 'State Code' and feature_name != 'FIPS Code' and feature_name != 'County Code':
    vals = df[feature_name].values
    counter = 0;
    currsum = 0;
    avg = 1;
    # calculate the average value of the column
    for i in range(len(vals)):
      if not pd.isna(vals[i]):
        counter+=1
        currsum+=vals[i]
    avg = currsum/counter
    #replace all NaNs with the average
    for i in range(len(vals)):
      if pd.isna(vals[i]):
        df[feature_name][i+1] = avg;
        vals = df[feature_name].values

df.head(3)

,State Code,County Code,FIPS Code,State Abbreviation,County Name,Premature Death,Poor or Fair Health,Poor Physical Health Days,Poor Mental Health Days,Low Birthweight,...,Percent 65 and Older,Percent Non-Hispanic Black,Percent American Indian & Alaska Native,Percent Asian,Percent Native Hawaiian/Other Pacific Islander,Percent Hispanic,Percent Non-Hispanic White,Percent Not Proficient in English,Percent Female,Percent Rural
1,1,1,1001,AL,Autauga County,1.677055,2.239968,2.681249,3.177324,3.359182,...,2.355468,1.410688,0.054278,0.398180,0.243168,0.179171,3.480834,0.263306,11.066949,1.344037
2,1,3,1003,AL,Baldwin County,1.707955,1.659976,2.225780,2.820540,2.696064,...,3.487629,0.586636,0.093580,0.375314,0.148372,0.289419,3.981838,0.293477,11.103740,1.352899
3,1,5,1005,AL,Barbour County,3.324352,4.439937,4.516030,4.182731,4.426371,...,3.210978,3.324059,0.083266,0.154990,0.455111,0.292215,2.102566,0.694653,9.111444,2.169216


**Create Prediction Algorithm**
1. loop through each metric(because we want a prediction algorithm for each metric)
2. split the data into training and test data(train the algorithm on 70% of the data, and use the other 30% to test the algorithm's accuracy)
3. create and train the kernel ridge regression with the training data(this is what predicts the value. It's make with the sklearn package.)

  input 86 metrics -> output 1 predicted value for the 87th metric

4.  use the kernel ridge regression algorithm on the test data to get the accuracy of the kernel ridge regression. **Accuracy is calculated with an explained variance score**


In [ ]:
#run the machine learning algorithm
predictor = {}
predictionAccuracies = {}
errors = []
#loop through each metric
for desiredmetric in df.columns:
  if desiredmetric != 'State Abbreviation' and desiredmetric != 'County Name' and desiredmetric != 'State Code' and desiredmetric != 'FIPS Code' and desiredmetric != 'County Code':
    #split data into predicted data and prediction metrics
    data_Y = df[desiredmetric]
    data_X = df.drop([desiredmetric,'County Name', 'State Abbreviation', 'FIPS Code', 'County Code', 'State Code'],axis=1)
    #split into training and testing data
    X_train, X_test, Y_train, Y_test = train_test_split(data_X, data_Y, test_size=0.3)

    #create and train kernel ridge regression algorithm
    krr = KernelRidge(alpha=1.0)
    krr.fit(X_train, Y_train)

    #use krr on test data and get the error
    preds = krr.predict(X_test)   
    error = explained_variance_score(Y_test, preds)

    #store the algorithm and errors
    predictor[desiredmetric] = krr
    predictionAccuracies[desiredmetric] = error
    errors.append(error)

    print('Accuracy for %s is: %.3f' % (desiredmetric, error))

print('Average Accuracy: %.3f' % (sum(errors)/len(errors)))

Accuracy for Premature Death is: 0.954
Accuracy for Poor or Fair Health is: 0.990
Accuracy for Poor Physical Health Days is: 0.994
Accuracy for Poor Mental Health Days is: 0.989
Accuracy for Low Birthweight is: 0.677
Accuracy for Adult Smoking is: 0.939
Accuracy for Adult Obesity is: 0.808
Accuracy for Food Environment Index is: 0.987
Accuracy for Physical Inactivity is: 0.897
Accuracy for Access to Exercise Opportunities is: 0.534
Accuracy for Excessive Drinking is: 0.709
Accuracy for Alcohol-Impaired Driving Deaths is: 0.070
Accuracy for Sexually-Transmitted Infections is: 0.761
Accuracy for Teen Births is: 0.775
Accuracy for Uninsured is: 0.999
Accuracy for Primary Care Physicians is: 0.516
Accuracy for Dentists is: 0.418
Accuracy for Mental Health Providers is: 0.342
Accuracy for Preventable Hospital Stays is: 0.384
Accuracy for Mammography Screening is: 0.594
Accuracy for Flu Vaccinations is: 0.522
Accuracy for High School Completion is: 0.836
Accuracy for Some College is: 0.779
A

**Health Score Algorithm**

Take a county

Average the QOL metrics

Average the LOL metrics

Sum it up for a health score

In [ ]:
#Define health score calculations and metric increase/decreases
#Calculate the health score. The lower the health score, the better
def healthscore(df, row):
  #these metrics are 50% of the health score
  healthScoreQOLMetrics = ['Poor or Fair Health', 'Poor Physical Health Days', 'Poor Mental Health Days', 
                              'Frequent Physical Distress', 'Frequent Mental Distress', 'Diabetes Prevalence']
  #these metrics are 50% of the health score
  healthScoreLOLMetrics = ['Premature Death', 'Premature Age-Adjusted Mortality']
  
  countydata = df.iloc[[row]]
  QOLsum = 0
  LOLsum = 9-countydata['Life Expectancy']/len(healthScoreLOLMetrics)
  for column in healthScoreQOLMetrics:
    QOLsum += countydata[column]
  for column in healthScoreLOLMetrics:
    LOLsum += countydata[column]
  #use the average value to weigh QOL and LOL 50/50
  score = QOLsum/len(healthScoreQOLMetrics) + LOLsum/len(healthScoreLOLMetrics)
  return score

#calculate all the health scores in a dataframe
def healthscores(df):
  healthscores = []
  for x in range(0, len(df['Premature Death'])):
    healthscores.append(healthscore(df, x).values[0])
  return healthscores


#Increase/decrease the metrics given to the function by a set amount
def varyHealthMetrics(df, metrics, amount):
  result = df.copy()
  for feature_name in df:
    if feature_name != 'State Abbreviation' and feature_name != 'County Name' and feature_name != 'State Code' and feature_name != 'FIPS Code' and feature_name != 'County Code':
      if feature_name in metrics:
        result[feature_name] = result[feature_name]+amount
  return result

neghealthmetriclist=['Severe Housing Cost Burden', 'Traffic Volume', 'Long Commute - Driving Alone', 
                  'Driving Alone to Work', 'Severe Housing Problems', 'Drinking Water Violations', 'Air Pollution - Particulate Matter', 
                  'Juvenile Arrests', 'Firearm Fatalities', 'Suicides', 'Homicides', 'Injury Deaths', 'Violent Crime', 'Childcare Cost Burden', 
                  'Residential Segregation - Black/White', 'Residential Segregation - Non-White/White', 'Children in Poverty', 'Income Inequality', 
                  'Children in Single-Parent Households', 'Gender Pay Gap', 'Children Eligible for Free or Reduced Price Lunch', 
                  'Unemployment', 'Disconnected Youth', 'School Segregation', 'Preventable Hospital Stays', 'Uninsured', 'Uninsured Adults', 'Uninsured Children', 
                  'Sexually-Transmitted Infections', 'Teen Births', 'Excessive Drinking', 'Alcohol-Impaired Driving Deaths', 'Drug Overdose Deaths', 
                  'Motor Vehicle Crash Deaths', 'Adult Obesity', 'Physical Inactivity', 'Food Insecurity', 'Limited Access to Healthy Foods', 
                  'Adult Smoking']
poshealthmetriclist=['Broadband Access', 'Homeownership', 'Childcare Centers', 'Social Associations', 'Median Household Income', 'High School Completion', 
                     'Some College', 'High School Graduation', 'Reading Scores', 'Math Scores', 'School Funding Adequacy', 'Flu Vaccinations', 
                     'Mammography Screening', 'Primary Care Physicians', 'Dentists', 'Mental Health Providers', 
                     'Other Primary Care Providers', 'Access to Exercise Opportunities', 'Food Environment Index']

difference = .5
#betterdf = varyHealthMetrics(df, neghealthmetriclist, -difference)
#betterdf = varyHealthMetrics(betterdf, poshealthmetriclist, difference)

#worsedf = varyHealthMetrics(df, neghealthmetriclist, difference)
#worsedf = varyHealthMetrics(worsedf, poshealthmetriclist, -difference)
currentHS = healthscores(df)

**Find The Counties**

1. Calculate current(actual) health scores of each county

2. Create hypothetical counties with an improved metric(by .5)

3. Use the kernel ridge regression to predict new health score metrics using the improved metric. 

We are predicting ['Poor or Fair Health', 'Poor Physical Health Days', 'Poor Mental Health Days', 'HIV Prevalence', 
                      'Low Birthweight', 'Frequent Physical Distress', 'Frequent Mental Distress', 'Diabetes Prevalence', 
                      'Premature Death', 'COVID-19 Age-Adjusted Mortality', 'Premature Age-Adjusted Mortality', 'Child Mortality'] 

using the hypothetical county data(with one improved metric)

4. Using the new predicted health score metrics, calculate a improved health score for each county

5. Compare the improved health score to the actual health score. Top 5 most improved is best counties to allocate resources to, Top 5 least improved is worst counties to allocate resources to. 

In [ ]:
#List counties with best improvements
healthScoreMetrics = ['Poor or Fair Health', 'Poor Physical Health Days', 'Poor Mental Health Days', 'HIV Prevalence', 
                      'Low Birthweight', 'Frequent Physical Distress', 'Frequent Mental Distress', 'Diabetes Prevalence', 
                      'Premature Death', 'COVID-19 Age-Adjusted Mortality', 'Premature Age-Adjusted Mortality', 'Child Mortality']

#metrics = ['Primary Care Physicians', 'Mental Health Providers', 'Flu Vaccinations', 'Sexually-Transmitted Infections']
metrics = ['Children in Poverty']
HSimprovements = {}
HScounties = {}
for metric in metrics: 
  posdf = varyHealthMetrics(df, [metric], 1)
  for desiredmetric in healthScoreMetrics:
    #create better dataframes
    betterdata_X = posdf.drop([desiredmetric, 'County Name', 'State Abbreviation', 'FIPS Code', 'County Code', 'State Code'],axis=1)

    #predict the desiredmetric using the new dataframtes
    betterPredictedVals = predictor[desiredmetric].predict(betterdata_X)

    #set the metric to the predicted metric and calculate hypothetical healthscores
    posdf[desiredmetric] = betterPredictedVals
  betterHS = healthscores(posdf)


  #calculate the difference in health scores for each county
  improvedHSCounty = {}
  improvedHS = []
  for i in range(len(currentHS)):
    currentcounty = df['County Name'][i+1] + ', ' + df['State Abbreviation'][i+1]
    improvedHSCounty[currentHS[i]-betterHS[i]] = currentcounty
    improvedHS.append(currentHS[i]-betterHS[i])

  #print the best and worst health score differences
  improvedHS.sort()
  HScounties[metric] = improvedHSCounty
  HSimprovements[metric] = improvedHS

  goodcountynames = []
  badcountynames = []
  print(metric)
  for i in range(1,11):
    print(str(i) + '. ' + improvedHSCounty[improvedHS[i]] + " by %.3f Health Score " % (-improvedHS[i]))
    goodcountynames.append(improvedHSCounty[improvedHS[i]])
  for i in range(len(improvedHS)-10,len(improvedHS)): 
    print(str(i) + '. ' + improvedHSCounty[improvedHS[i]] + " by %.3f Health Score " % (-improvedHS[i]))
    badcountynames.append(improvedHSCounty[improvedHS[i]])
  print()

Children in Poverty
1. Prairie County, MT by 2.355 Health Score 
2. Briscoe County, TX by 2.218 Health Score 
3. Nelson County, ND by 2.174 Health Score 
4. Yukon-Koyukuk Census Area, AK by 2.130 Health Score 
5. Huerfano County, CO by 2.060 Health Score 
6. Sutton County, TX by 2.055 Health Score 
7. Hidalgo County, NM by 2.055 Health Score 
8. Leslie County, KY by 2.045 Health Score 
9. North Slope Borough, AK by 2.030 Health Score 
10. Wallowa County, OR by 2.028 Health Score 
3072. Stonewall County, TX by 0.802 Health Score 
3073. Stanley County, SD by 0.766 Health Score 
3074. Oliver County, ND by 0.752 Health Score 
3075. Franklin County, NE by 0.747 Health Score 
3076. Motley County, TX by 0.717 Health Score 
3077. Dewey County, SD by 0.488 Health Score 
3078. Taliaferro County, GA by 0.337 Health Score 
3079. Mono County, CA by 0.334 Health Score 
3080. Edwards County, KS by 0.316 Health Score 
3081. Alpine County, CA by -0.457 Health Score 



**Find Reasons Why One County Is Affected More Than Another**
1. Average the values of each metric(all 87) for the top 5 counties and bottom 5 counties

2. Compare the average values of the top 5 and bottom 5

3. Top 5 metrics with the biggest difference in average value are the top 5 reasons why one county is affected more than another. 

In [ ]:
goodcounties = []
badcounties = []
for i in range(len(df['County Name'])):
  for a in range(len(goodcountynames)):
    if df.iloc[i][4] in goodcountynames[a] and df.iloc[i][3] in goodcountynames[a]:
      goodcounties.append(df.iloc[[i]])
  for a in range(len(badcountynames)):
    if df.iloc[i][4] in badcountynames[a] and df.iloc[i][3] in badcountynames[a]:
      badcounties.append(df.iloc[[i]])
def avgval(counties, metric):
  sum = 0
  for county in counties:
    sum+=county[metric].values[0]
  return sum/len(counties)
goodaverages = []
badaverages = []
metrics = ['Premature Death', 'Poor or Fair Health', 'Poor Physical Health Days', 'Poor Mental Health Days', 
                'Low Birthweight', 'Adult Smoking', 'Adult Obesity', 'Food Environment Index', 'Physical Inactivity', 
                'Access to Exercise Opportunities', 'Excessive Drinking', 'Alcohol-Impaired Driving Deaths', 'Sexually-Transmitted Infections', 'Teen Births', 
                'Uninsured', 'Primary Care Physicians', 'Dentists', 'Mental Health Providers', 'Preventable Hospital Stays', 
                'Mammography Screening', 'Flu Vaccinations', 'High School Completion', 'Some College', 'Unemployment', 
                'Children in Poverty', 'Income Inequality', 'Children in Single-Parent Households', 'Social Associations', 
                'Violent Crime', 'Injury Deaths', 'Air Pollution - Particulate Matter', 'Drinking Water Violations', 
                'Severe Housing Problems', 'Percentage of Households With High Housing Costs', 'Driving Alone to Work', 
                'Long Commute - Driving Alone', 'COVID-19 Age-Adjusted Mortality', 'Life Expectancy', 'Premature Age-Adjusted Mortality', 
                'Child Mortality', 'Frequent Physical Distress', 'Frequent Mental Distress', 'Diabetes Prevalence', 'HIV Prevalence', 'Food Insecurity', 
                'Limited Access to Healthy Foods', 'Drug Overdose Deaths', 'Motor Vehicle Crash Deaths', 'Insufficient Sleep', 
                'Uninsured Adults', 'Uninsured Children', 'Other Primary Care Providers', 'High School Graduation', 'Disconnected Youth', 
                'Reading Scores', 'Reading Scores(Black)', 'Reading Scores(Hispanic)', 'Reading Scores(White)', 'Math Scores', 
                'School Segregation', 'School Funding Adequacy', 'Gender Pay Gap', 'Median Household Income', 'Children Eligible for Free or Reduced Price Lunch', 
                'Residential Segregation - Black/White', 'Residential Segregation - Non-White/White', 'Childcare Cost Burden', 
                'Childcare Centers', 'Homicides', 'Suicides', 'Firearm Fatalities', 'Juvenile Arrests', 'Traffic Volume', 'Homeownership', 
                'Severe Housing Cost Burden', 'Broadband Access', 'Population', 'Percent Below 18 Years of Age', 'Percent 65 and Older', 
                'Percent Non-Hispanic Black', 'Percent American Indian & Alaska Native', 'Percent Asian', 'Percent Native Hawaiian/Other Pacific Islander', 
                'Percent Hispanic', 'Percent Non-Hispanic White', 'Percent Not Proficient in English', 'Percent Female', 'Percent Rural']
for metric in metrics: 
  goodaverages.append(avgval(goodcounties, metric))
  badaverages.append(avgval(badcounties, metric))
  diff = avgval(goodcounties, metric) - avgval(badcounties, metric)
  if abs(diff) > 1:
    print(metric + ' diff is %.3f' % diff)


Premature Death diff is -2.612
Food Environment Index diff is -1.016
Severe Housing Problems diff is 1.923
Driving Alone to Work diff is -1.656
Premature Age-Adjusted Mortality diff is -1.305
Gender Pay Gap diff is 1.290
